In [8]:
import pandas as pd

In [9]:
import re

# Data

In [17]:
words = pd.read_csv('data/wikidata_results.tsv','\t')

/tmp/ipykernel_115831/2847847135.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  words = pd.read_csv('data/wikidata_results.tsv','\t')


In [18]:
words

,entity,entityLabel,entity_male_label,entity_female_label
0,http://www.wikidata.org/entity/Q76477,neveu ou nièce,neveu,nièce
1,http://www.wikidata.org/entity/Q76666,cousin germain,cousin germain,cousine germaine
2,http://www.wikidata.org/entity/Q80687,secrétaire,secrétaire,secrétaire
3,http://www.wikidata.org/entity/Q81096,ingénieur ou ingénieure,ingénieur,ingénieure
4,http://www.wikidata.org/entity/Q82594,informaticien ou informaticienne,informaticien,informaticienne
...,...,...,...,...
52148,http://www.wikidata.org/entity/Q45199,marin,NaN,marine
52149,http://www.wikidata.org/entity/Q45713,Titan,NaN,titanide
52150,http://www.wikidata.org/entity/Q54128,domestique,NaN,domestique
52151,http://www.wikidata.org/entity/Q59163,pilier,NaN,pilière


In [19]:
where = words.entity_male_label.isna()
words.entity_male_label[where] = words.entityLabel[where]

In [20]:
from collections import Counter

In [21]:
masculine2feminine = {}
for _, word in words.iterrows():
    masculine = word.entity_male_label.lower().strip()
    masculine2feminine.setdefault(masculine, Counter())
    feminine = word.entity_female_label.lower().strip()
    masculine2feminine[masculine][feminine]+=1

In [22]:
for m, fs in masculine2feminine.items():
    masculine2feminine[m] = fs.most_common(1)[0][0]     
    

def desc(masculine2feminine):
    sorted_m2f = {}
    for m in sorted(masculine2feminine, key=len, reverse=True):
        sorted_m2f[m] = masculine2feminine[m]
    return sorted_m2f

masculine2feminine = desc(masculine2feminine)

masculine2feminine

## Analysis

In [116]:
m_len,f_len=[],[]#set(),set()
for m, f in masculine2feminine.items():
    l = len(model(m))
   # if l not in m_len:
      #  print(l, m,f)
      #  m_len.add(l)
    m_len.append(len(model(m)))
    f_len.append(len(model(f)))

In [117]:
import seaborn as sns

In [118]:
pd.DataFrame(m_len).describe()

,0
count,48648.000000
mean,4.319191
std,1.343991
min,1.000000
25%,4.000000
50%,4.000000
75%,4.000000
max,24.000000


In [122]:
fig=sns.displot(m_len,discrete=True)

In [123]:
fig.savefig("dis_m_len.png")

# Translation

def translate_replace(text):
    text = text.strip().lower()
    text = re.sub(r'\s+',' ',text)
    for m, f in masculine2feminine.items():
        m = re.sub(r"[\(\)]","",m).strip()
        old_text = text
        text = re.sub(fr"\b{m}\b",f,text)#text.replace(m,f)#
        if text!= old_text:
            print(m,f)
    return text

In [3]:
from pathlib import Path

In [1]:
import spacy

/vol/work/lerner/anaconda3/envs/datasets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-22 16:19:44.477782: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-22 16:19:44.477802: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-09-22 16:19:50.388223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-22 16:19:50.389007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic li

In [2]:
model = spacy.load("fr_dep_news_trf")

In [67]:
from spacy import displacy
displacy.serve(model(text), style="dep")

/vol/work/lerner/anaconda3/envs/datasets/lib/python3.10/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [4]:
standard_root = Path('data/corpus-francais-inclusif/vfs/')

In [71]:
for path in standard_root.glob('*.txt'):
    print(path)
    with open(path, 'rt') as file:
        text = file.read()
    print(text)
    print("\t",translate(text))
    print("\n***************\n")

data/corpus-francais-inclusif/vfs/coord-1974-giscard.txt
Martiniquaises, Martiniquais, me voici donc à la Martinique où personne n'est jamais venu sans aimer à la fois cette terre et ce peuple. Voici donc les Martiniquaises et les Martiniquais rassemblés pour accueillir le President de la République qu'ils ont élu eux-mêmes démocratiquement.

https://www.vie-publique.fr/discours/262820-allocution-de-m-valery-giscard-destaing-place-de-la-savane-fort-de-f

	 martiniquaises , martiniquais , me voici donc à la martinique où personne n' est jamais venu sans aimer à la fois cette terre et ce peuple . voici donc les martiniquaises et les martiniquais rassemblés pour accueillir le president de la république qu' ils ont élu eux - mêmes démocratiquement . 

 


***************

data/corpus-francais-inclusif/vfs/coord-1981-giscard.txt
Mes chers amis, ceux qui sont à l'intérieur, ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation

In [72]:

with open("data/corpus-francais-inclusif/vfs/coord-1981-giscard.txt", 'rt') as file:
    text = file.read()
print(text)
    

Mes chers amis, ceux qui sont à l'intérieur, ceux qui sont au dehors, car je m'excuse auprès d'eux, le chapiteau était trop petit, mais j'espère que la sonorisation leur permet de nous entendre... mes chers amis, j'étais à Lille il y a quinze jours et j'avais dit que j'essaierais de revenir dans le Pas-de-Calais.

Les Françaises et les Français m'ont placé en tête de dix candidats.

C'est pourquoi notre société doit être capable - et j'en prends l'engagement - d'offrir à toutes les jeunes Françaises et à tous les jeunes Français, soit un poste de travail, soit un poste de formation...

Mais regardez quelques réalités simples : d'abord, la consommation, nous venons de dépasser, cela a paru dans les journaux, mais c'était un entrefilet à peine lisible, le niveau de consommation individuelle des Américaines et des Américains : la consommatrice française, le consommateur français consomment un peu plus que l'Américaine ou l'Américain. Vous avez vu arriver, les plus anciens d'entre vous com

In [79]:
tokens = model(text)

In [83]:
def is_masc(token):
    gender = token.morph.get("Gender")
    return gender and gender[0] == "Masc" 

In [82]:
tokens[0].morph

Number=Plur|Poss=Yes

In [90]:
print(translate(text))

mes cher.es amis et amies , ceu.es qui sont à l' intérieur , ceu.es qui sont au dehors , car je m' excuse auprès d' eux , le chapiteau était trop petit.e , mais j' espère que la sonorisation leur permet de nous entendre ... mes cher.es amis et amies , j' étais à lille il y a quinze jours et j' avais dit que j' essaierais de revenir dans le pas-de-calais . 

 les françaises et les français et françaises m' ont placé en tête de dix candidats et candidates . 

 c' est pourquoi notre société doit être capable - et j' en prends l' engagement - d' offrir à toutes les jeunes françaises et à tou.es les jeunes français et françaises , soit un poste.e de travail , soit un poste de formation ... 

 mais regardez quelques réalités simples : d' abord , la consommation , nous venons de dépasser , cela a paru dans les journaux , mais c' était un entrefilet à peine lisible , le niveau de consommation individuelle des américaines et des américains : la consommatrice française , le.e consommateur et con

In [47]:
token.morph.get("Gender")

['Fem']

In [89]:
def is_plur(token):    
    num = token.morph.get("Number")
    return num and num[0] == "Plur"

In [87]:
def translate(text):
    translation = []
    tokens = model(text)
    todo = set()
    for token in tokens:
       # if token.like_url:
        #    continue
        if not is_masc(token):            
            translation.append(token.lower_)
            continue
        lemma = token.lemma_
        if lemma in masculine2feminine:
            for child in token.children:
                todo.add(child.i)
            f = masculine2feminine[lemma]
            if is_plur(token):
                f+="s"
            translation.append(f"{token.lower_} et {f}")
        else:
            translation.append(token.lower_)
    for i in todo:
        token = tokens[i]
        if is_masc(token):
            if is_plur(token):
                translation[i] = translation[i][:-1]+".es"
            else:
                translation[i]+=".e"
    return " ".join(translation)

In [56]:
token.lower_ in masculine2feminine

False

In [39]:
cat data/corpus-francais-inclusif/vfs/coord-1981-mitterrand.txt

Tous ceux que je n'ai pu voir au-cours de cette brève visite, ceux qui tiennent les stands, ceux qui organisent les visites, ceux qui ont conçu, organisé, ont droit à la reconnaissance du pays dans la mesure où ils nous fournissent et lui fournissent l'occasion de rassembler en quelques jours et dans un endroit bien circonscrit un ensemble de réalisations qui marquent la capacité de la France.

https://www.vie-publique.fr/discours/134914-allocution-de-m-francois-mitterrand-president-de-la-republique-loc


In [ ]:
!tail -n 1 data/corpus-francais-inclusif/vfs/*txt

==> data/corpus-francais-inclusif/vfs/coord-1974-giscard.txt <==
https://www.vie-publique.fr/discours/262820-allocution-de-m-valery-giscard-destaing-place-de-la-savane-fort-de-f

==> data/corpus-francais-inclusif/vfs/coord-1981-giscard.txt <==
https://www.vie-publique.fr/discours/136564-discours-de-m-valery-giscard-destaing-president-de-la-republique-not

==> data/corpus-francais-inclusif/vfs/coord-1981-mitterrand.txt <==
https://www.vie-publique.fr/discours/134914-allocution-de-m-francois-mitterrand-president-de-la-republique-loc

==> data/corpus-francais-inclusif/vfs/coord-1987-mitterrand2.txt <==
https://www.vie-publique.fr/discours/139391-mitterrand-23051987-place-la-revolution-francaise-de-1789-lormont

==> data/corpus-francais-inclusif/vfs/coord-1987-mitterrand.txt <==
https://www.vie-publique.fr/discours/139776-discours-de-m-francois-mitterrand-president-de-la-republique-devant-l

==> data/corpus-francais-inclusif/vfs/coord-1989-mitterrand.txt <==
https://www.vie-publique.fr/dis